#### This is mock data that is created to apply the predictions of the model

The key thing to take away from the model is that it is a means to "run" the comp or play the comp without actually playing the game. We aggregate how specific team compositions perform against the field and use this information to see if we can derive some information of comps we haven't played or have not played as often. 

Here we focus on the knights build for simplicity of creating viable data with the following assumptions:
- All support units will be tier 2
- There is a cap of 8 hero's on the field so other than the extra carry champion (other than Galio and Garen) there is possible an extra champion for synergy sake
- We will consider all permutation of carry heros and tiers of 1 and 2. 

In [22]:
import json
import numpy as np
import pandas as pd
import csv
import time
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import itertools

In [2]:
data = pd.read_pickle("synergy_carry_value_df_redux.pkl")

In [18]:
data_knight = data[data["Set5_Knight"] == "gold"]
data_knight = data_knight[(data_knight["Primary_Carry"] == "TFT5_Galio") & (data_knight["Secondary_Carry"] == "TFT5_Garen")]

In [54]:
base = dict(data_knight.iloc[0])
base['Set5_Ironclad'] = "grey"
base['Set5_Redeemed'] = "grey"
base['Set5_Cavalier'] = "grey"
base['Value'] = 18
base['Number of Bronze'] = 0
base

{'Set5_Assassin': 'grey',
 'Set5_Brawler': 'grey',
 'Set5_Cavalier': 'grey',
 'Set5_Forgotten': 'grey',
 'Set5_Ironclad': 'grey',
 'Set5_Legionnaire': 'grey',
 'Set5_Nightbringer': 'grey',
 'Set5_Redeemed': 'grey',
 'Set5_Sentinel': 'grey',
 'Set5_Skirmisher': 'grey',
 'Set5_Dawnbringer': 'grey',
 'Set5_Invoker': 'grey',
 'Set5_Abomination': 'grey',
 'Set5_Knight': 'gold',
 'Set5_Hellion': 'grey',
 'Set5_Ranger': 'grey',
 'Set5_Spellweaver': 'grey',
 'Set5_Cannoneer': 'grey',
 'Set5_Revenant': 'grey',
 'Set5_Draconic': 'grey',
 'Set5_Mystic': 'grey',
 'Set5_Renewer': 'grey',
 'Number of Chromatic': 0,
 'Number of Gold': 1,
 'Number of Silver': 0,
 'Number of Bronze': 0,
 'Synergy Contest': 'Contested',
 'Vertical Synergies': False,
 'Primary_Carry_Tier': 2,
 'Primary_Carry': 'TFT5_Galio',
 'Secondary_Carry_Tier': 1,
 'Secondary_Carry': 'TFT5_Garen',
 'Third_Carry_Tier': 1,
 'Third_Carry': 'TFT5_Kayle',
 'Carry_Contested': 'Contested',
 'Value': 18,
 'Diff_from_Match_Max': 40,
 'Last_Ro

In [126]:
carry_champs = ['TFT5_Akshan', 'TFT5_Aphelios', 'TFT5_Draven', 'TFT5_Gwen', 'TFT5_Heimerdinger', 'TFT5_Jax', 'TFT5_Karma', 
               'TFT5_Kayle', 'TFT5_Lucian', 'TFT5_Teemo', 'TFT5_Velkoz']
four_cost = ['TFT5_Galio','TFT5_Aphelios','TFT5_Draven','TFT5_Jax','TFT5_Karma','TFT5_Lucian','TFT5_Velkoz']
total_list = []

In [127]:
## Assumption with Kayle the standard build would be Rell which would give the bronze ironclad, 
## bronze cavalier and bronze redeemed. So we assume there is a tier 2 Rell
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Kayle']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])

kayle_list = []
current_base = base.copy()
current_base_value = 30
current_base['Set5_Ironclad'] = "bronze"
current_base['Set5_Redeemed'] = "bronze"
current_base['Set5_Cavalier'] = "bronze"
current_base['Number of Bronze'] = 3
for i in range(len(carries)):
    temp = kayle_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    temp["Value"] = current_base_value + added_value
    kayle_list.append(temp)
total_list += kayle_list

In [128]:
## Akshan Carry Gives two possible synergies, sentinel or ranger. We will capture both
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Akshan']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
akshan_list = []
## We will assume sentinels and a reasonable choice is rakan a 3 cost hero
current_base = base.copy()
current_base_value = 27
current_base['Set5_Sentinel'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    akshan_list.append(temp)

## We will assume here rangers and a reasonable choice is aphelios a 3 cost hero
current_base = base.copy()
current_base_value = 30
current_base['Set5_Ranger'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    temp["Value"] = current_base_value + added_value
    akshan_list.append(temp)
total_list += akshan_list

In [129]:
## Aphelios Carry Gives two possible synergies, nightbringer or ranger. We will capture both
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Aphelios']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
aphelios_list = []
## We will assume nightbringer and a reasonable choice is diana a 4 cost hero
current_base = base.copy()
current_base_value = 30
current_base['Set5_Nightbringer'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    aphelios_list.append(temp)

## We will assume here rangers and a reasonable choice is akshan a 5 cost hero
current_base = base.copy()
current_base_value = 33
current_base['Set5_Ranger'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    temp["Value"] = current_base_value + added_value
    aphelios_list.append(temp)
total_list += aphelios_list

In [130]:
## Draven Carry Gives two possible synergies, Legionnaire or Forgotten. We will choose Kayle
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Draven']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
draven_list = []
current_base = base.copy()
current_base_value = 33
current_base['Set5_Legionnaire'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    draven_list.append(temp)
total_list += draven_list

In [131]:
## Heimerdinger Carry Gives 1 possible synergies, Renewer and so we choose Ivan a 4 cost hero
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Heimerdinger']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
heimer_list = []
current_base = base.copy()
current_base_value = 30
current_base['Set5_Renewer'] = "bronze"
current_base['Number of Bronze'] = 1
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    heimer_list.append(temp)
total_list += heimer_list

In [132]:
## Karma Carry Gives 2 synergies, dawnbringer and invoker we choose teemo a 4 cost hero
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Karma']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
karma_list = []
current_base = base.copy()
current_base_value = 33
current_base['Set5_Dawnbringer'] = "bronze"
current_base['Set5_Invoker'] = "bronze"
current_base['Number of Bronze'] = 2
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    karma_list.append(temp)
total_list += karma_list

In [133]:
## Teemo Carry Gives 1 synergies, invoker we choose Karma a 4 cost hero to give dawnbringer too
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Teemo']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
teemo_list = []
current_base = base.copy()
current_base_value = 30
current_base['Set5_Dawnbringer'] = "bronze"
current_base['Set5_Invoker'] = "bronze"
current_base['Number of Bronze'] = 2
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    teemo_list.append(temp)
total_list += teemo_list

In [134]:
## Lucian Carry Gives 2 synergies with addition of Senna, Sentinel and Cannoneer
champions = ['TFT5_Galio', 'TFT5_Garen', 'TFT5_Lucian']
champions_perm = []
for i in itertools.permutations(champions):
    champions_perm.append(list(i))
tiers_perm = tuple([x,y,z] for x in [1,2] for y in [1,2] for z in [1,2])
carries = []
for i,j in itertools.product(tiers_perm,champions_perm):
    carries.append([i,j])
    
lucian_list = []
current_base = base.copy()
current_base_value = 21
current_base['Set5_Sentinel'] = "bronze"
current_base['Set5_Cannoneer'] = "bronze"
current_base['Number of Bronze'] = 2
for i in range(len(carries)):
    temp = current_base.copy()
    temp["Primary_Carry_Tier"] = carries[i][0][0]
    temp['Primary_Carry'] = carries[i][1][0]
    temp['Secondary_Carry_Tier'] = carries[i][0][1]
    temp['Secondary_Carry'] = carries[i][1][1]
    temp['Third_Carry_Tier'] = carries[i][0][2]
    temp['Third_Carry'] = carries[i][1][2]
    value_carry = []
    for x in carries[i][1]:
        if x in four_cost:
            value_carry.append(4)
        else:
            value_carry.append(5)
    added_value = 0
    for j in range(3):
        added_value += 3**(carries[i][0][j]-1) * value_carry[j]
    
    temp["Value"] = current_base_value + added_value
    lucian_list.append(temp)
total_list += lucian_list

In [135]:
len(total_list)

480

In [137]:
mock_data = pd.DataFrame(total_list)

In [138]:
mock_data.to_pickle("mock_data.pkl")

In [143]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mock_data.iloc[:,-12:-6]

,Primary_Carry_Tier,Primary_Carry,Secondary_Carry_Tier,Secondary_Carry,Third_Carry_Tier,Third_Carry
0,1,TFT5_Galio,1,TFT5_Garen,1,TFT5_Kayle
1,1,TFT5_Galio,1,TFT5_Kayle,1,TFT5_Garen
2,1,TFT5_Garen,1,TFT5_Galio,1,TFT5_Kayle
3,1,TFT5_Garen,1,TFT5_Kayle,1,TFT5_Galio
4,1,TFT5_Kayle,1,TFT5_Galio,1,TFT5_Garen
5,1,TFT5_Kayle,1,TFT5_Garen,1,TFT5_Galio
6,1,TFT5_Galio,1,TFT5_Garen,2,TFT5_Kayle
7,1,TFT5_Galio,1,TFT5_Kayle,2,TFT5_Garen
8,1,TFT5_Garen,1,TFT5_Galio,2,TFT5_Kayle
9,1,TFT5_Garen,1,TFT5_Kayle,2,TFT5_Galio
